In [1]:
% load_ext autoreload
% autoreload 2

In [2]:
import utils
import train
import cvae

In [3]:
train_iter, val_iter, test, DE, EN = utils.torchtext_extract()

In [4]:
model_name = "test_attention_posterior"

gpu = True

num_layers = 2
embed_size = 100
hidden_size = 100
latent_size = 100

num_epochs=10

anneal = utils.kl_anneal_linear

eos = EN.vocab.stoi["</s>"]
bos = EN.vocab.stoi["<s>"]
pad = EN.vocab.stoi["<pad>"]

filter_beam = [pad, bos]

In [5]:
model = cvae.CVAE(len(DE.vocab), len(EN.vocab), embed_size, hidden_size, latent_size, num_layers)
if gpu:
    model.cuda()
print("Number of parameters: {}".format(utils.count_parameters(model)))

Number of parameters: 10746960


/home/kevin/.local/lib/python3.5/site-packages/torch/backends/cudnn/__init__.py:84: UserWarning: PyTorch was compiled without cuDNN support. To use cuDNN, rebuild PyTorch making sure the library is visible to the build system.
  "PyTorch was compiled without cuDNN support. To use cuDNN, rebuild "


In [6]:
train.train(model,model_name, train_iter, val_iter, DE, EN, anneal, num_epochs, gpu)

  0%|          | 0/3722 [00:00<?, ?it/s]/home/kevin/.local/lib/python3.5/site-packages/torch/backends/cudnn/__init__.py:84: UserWarning: PyTorch was compiled without cuDNN support. To use cuDNN, rebuild PyTorch making sure the library is visible to the build system.
  "PyTorch was compiled without cuDNN support. To use cuDNN, rebuild "
  0%|          | 0/18 [00:00<?, ?it/s]/usr/local/lib/python3.5/dist-packages/torchtext/data/field.py:321: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  return Variable(arr, volatile=not train)
  0%|          | 0/3722 [00:00<?, ?it/s]

Epoch: 1
	VPB: 1879220055561120906546408112097776314820272415281094017170018142609957035830626338027055949518954132986727857513724484752536744883155398879544608817152.0000 VNELBO: 346.0186 VRE: 3.5378 TKL: 342.4808
	TPB: 15263752125068862953892645668677318683317268293615694809786227487367453786258268657098599534619095857352560082944.0000 TNELBO: 258.3124 TRE: 4.3819 VKL: 253.9305


100%|██████████| 18/18 [03:27<00:00, 11.53s/it]
/home/kevin/.local/lib/python3.5/site-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 3-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)
/home/kevin/.local/lib/python3.5/site-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 2-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)
  0%|          | 0/3722 [00:00<?, ?it/s]

Epoch: 2
	VPB: 29.0240 VNELBO: 3.3681 VRE: 3.3457 TKL: 0.0224
	TPB: 70.8935 TNELBO: 4.2612 TRE: 3.6901 VKL: 0.5711
	BLEU: 0.0936


  0%|          | 1/3722 [00:00<12:13,  5.07it/s]

Epoch: 3
	VPB: 23.6789 VNELBO: 3.1646 VRE: 3.1566 TKL: 0.0080
	TPB: 27.7857 TNELBO: 3.3245 TRE: 3.3102 VKL: 0.0143


  0%|          | 0/3722 [00:00<?, ?it/s]

Epoch: 4
	VPB: 21.0143 VNELBO: 3.0452 VRE: 3.0361 TKL: 0.0091
	TPB: 21.5859 TNELBO: 3.0720 TRE: 3.0644 VKL: 0.0077
	BLEU: 0.1042


  4%|▎         | 139/3722 [00:20<08:52,  6.72it/s]

KeyboardInterrupt: 

In [ ]:
perp, elbo, nre, kl = utils.eval_vae(model, val_iter, pad, gpu=gpu)
bleu, _ = utils.test_generation(model, val_iter, EN, gpu=gpu)
print(perp, elbo, nre, kl, bleu)